In [1]:
import numpy as np
    # taxi data
    def latlng_to_meters(table, 
                         lat_pickup='pickup_latitude', lng_pickup='pickup_longitude', 
                         lat_dropoff='dropoff_latitude', lng_dropoff='dropoff_longitude'):
        # tag default shift
        origin_shift = 2 * np.pi * 6378137 / 2.0
        # tag columns we're not doing anything with 
        base_columns = '''VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, passenger_count, 
                          trip_distance, RateCodeID, store_and_fwd_flag, payment_type, fare_amount, 
                          extra, mta_tax,  tip_amount, tolls_amount, improvement_surcharge, total_amount'''
        # convert lat/lng coords within focused area to meters and return w/ base columns as pickup/dropoff x/y 
        latlng_to_meters = f'''
                           select
                               TAN( (90 + {lat_pickup}) * {np.pi} / 360.0 ) AS pickup_y,
                               {lng_pickup} * {origin_shift} / 180.0 AS pickup_x,
                               TAN( (90 + {lat_dropoff}) * {np.pi} / 360.0 ) AS dropoff_y,
                               {lng_dropoff} * {origin_shift} / 180.0 AS dropoff_x,
                               {base_columns}
                           from 
                               {table}
                               where
                                       {lng_pickup} < -73.75
                                   and {lng_pickup} > -74.15

                                   and {lng_dropoff} < -73.75
                                   and {lng_dropoff} > -74.15

                                   and {lat_pickup} > 40.68
                                   and {lat_pickup} < 40.84

                                   and {lat_dropoff} > 40.68
                                   and {lat_dropoff} < 40.84
                                   '''  # bottom half focuses coords, top half converts to meters & renames
        # run query & output results
        gdf = bc.sql(latlng_to_meters)
        # convert y columns w/ log
        for col in ['pickup_y', 'dropoff_y']:
            gdf[col] = np.log(gdf[col]) / (np.pi / 180.0)
        return gdf

In [2]:
%%time
from blazingsql import BlazingContext
# start up BlazingSQL w/o memory pool
bc = BlazingContext(pool=False)

BlazingContext ready
CPU times: user 2.25 s, sys: 502 ms, total: 2.75 s
Wall time: 2.44 s


In [3]:
%%time
# copy csv download to new name (to use while downloading)
!cp yellow_tripdata_2015-01.csv test_taxi.csv

CPU times: user 72.9 ms, sys: 39 ms, total: 112 ms
Wall time: 7.82 s


In [4]:
%%time
# create nyc taxi table
bc.create_table('taxi_2015', '/home/jupyter-winston/turbo-telegram/test_taxi.csv', header=0)

CPU times: user 14 ms, sys: 0 ns, total: 14 ms
Wall time: 13.1 ms


In [5]:
%%time
bc.sql('select * from taxi_2015').tail(2)

CPU times: user 5.75 s, sys: 2.64 s, total: 8.39 s
Wall time: 7.9 s


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
12748984,1,2015-01-10 19:01:44,2015-01-10 19:17:03,1,1.3,-73.999565,40.738483,1,N,-73.981819,40.737652,1,10.5,0.0,0.5,2.25,0.0,0.3,13.55
12748985,1,2015-01-10 19:01:45,2015-01-10 19:07:33,1,0.7,-73.960350,40.766399,1,N,-73.968643,40.760777,2,5.5,0.0,0.5,0.00,0.0,0.3,6.30


In [6]:
# gdf = bc.sql('select * from taxi_2015')
# type(gdf['pickup_latitude'][1])

In [7]:
gdf = latlng_to_meters('taxi_2015')

In [9]:
gdf.head(100000).to_csv('data/test_bsql_converted_taxi.csv')